Questions 1-8 should be answered by building a 15-period binomial model whose parameters should be calibrated to a Black-Scholes geometric Brownian motion model with: $T = .25$ years, $S_{0} = 100$, $r = 2\%$,  $\sigma = 30\%$,

and a dividend yield of $c = 1\%$.

In [3]:
# First, convert Black-Scholes continuous parameters to analogous discrete Binomial model parameters
import math
T = .25
S0 = 100
r = 2e-2
sigma = 30e-2
c = 1e-2

n = 15

R_n = math.exp(r * T / n)
R_n_c = 1 + r*T/n - c*T/n
u_n = math.exp(sigma * math.sqrt(T/n))
d_n = 1 / u_n

In [4]:
R_n, R_n_c, u_n, d_n

(1.0003333888950623,
 1.0001666666666666,
 1.0394896104013376,
 0.9620105771080376)

In [6]:
# Risk-neutral probability
q_n = (math.exp((r - c) * T/n) - d_n) / (u_n - d_n)

q_n

0.4924700506245105

In [7]:
def choose(n, k):
    return math.factorial(n) / (math.factorial(n - k) * math.factorial(k))

In [56]:
def choose_binomial(S0, u, d, q, n):
    value_nodes = []
    risk_neutral_nodes = []
    
    for i in range(n+1):
        num_paths = choose(n, i)
        num_down = i
        num_up = n - num_down
        
        value = S0 * u**num_up * d**num_down
        risk_neutral = num_paths * (q**num_up * (1-q)**num_down) * S0
        value_nodes.append(value)
        risk_neutral_nodes.append(risk_neutral)
    
    return value_nodes, risk_neutral_nodes

In [132]:
from collections import defaultdict


class Node(object):
    
    def __init__(self, value=None):
        self.value = value
        self.values = defaultdict(lambda: None)
        self.derived_value = None
        self.futures_value = None
        self.parent_up = None
        self.parent_down = None
        self.up = None
        self.down = None


class Lattice(object):
    
    def __init__(self):
        self.layers = []
    

class BinomialModel(object):
    
    def __init__(self, S0, u, d, q, r, n):
        self.S0 = S0
        self.q = q
        self.u = u
        self.d = d
        self.r = r
        self.n = n
        self.lattice = Lattice()
        self.fill_lattice()
        
    def fill_lattice(self):
        self.lattice.layers = [[Node() for i in range(i+1)] for i in range(self.n+1)]
        self.lattice.layers[0][0].value = self.S0
        self.lattice.layers[0][0].values['stock'] = self.S0
        
        for layer in range(self.n):
            for node_idx, node in enumerate(self.lattice.layers[layer]):
                next_layer = self.lattice.layers[layer + 1]
                
                up_node = next_layer[node_idx]
                down_node = next_layer[node_idx + 1]
                node.up = up_node
                node.down = down_node
                
                up_node.value = node.value * self.u
                up_node.values['stock'] = up_node.value
                up_node.parent_up = node
                
                down_node.value = node.value * self.d
                down_node.values['stock'] = down_node.value
                down_node.parent_down = node
    
    def fill_futures_lattice(self):
        def get_exercise_value(node_value, k):
            if call:
                return max(node.value - k, 0)
            else:
                return max(k - node.value, 0)
            
        for node in self.lattice.layers[-1]:
            node.values['futures'] = node.value
        
        for layer_idx in range(self.n - 1, -1, -1):
            layer = self.lattice.layers[layer_idx]
            for node in layer:            
                node.values['futures'] = self.q * node.up.values['futures'] + (1-self.q) * node.down.values['futures'] 
    
    def american_option(self, k, call=True, futures=None):
        def get_exercise_value(node_value, k):
            if call:
                return max(node_value - k, 0)
            else:
                return max(k - node_value, 0)
        
        if futures:
            self.fill_futures_lattice()
            base_value = 'futures'
            last_idx = futures
        else:
            base_value = 'stock'
            last_idx = self.n
            
        
        for node in self.lattice.layers[last_idx]:
            node.values['option'] = get_exercise_value(node.values[base_value], k)
        
        early_exercise = last_idx
        for layer_idx in range(last_idx - 1, -1, -1):
            layer = self.lattice.layers[layer_idx]
            for node in layer:
                exercise_value = get_exercise_value(node.values[base_value], k)
                continue_value = (
                    (1 / self.r) * (self.q * node.up.values['option'] + (1-self.q) * node.down.values['option'])
                )
                if exercise_value > continue_value:
                    early_exercise = min(early_exercise, layer_idx)
                node.values['option'] = max(exercise_value, continue_value)
        
        return self.lattice.layers[0][0].values['option'], early_exercise
    

In [133]:
%pdb on
bm = BinomialModel(S0, u_n, d_n, q_n, R_n, n)

[node.value for node in bm.lattice.layers[15]]
#[node.value for node in bm.lattice.layers[-1]]
bm.american_option(k=110, call=True)
#[node.values['option'] for node in bm.lattice.layers[14]]


Automatic pdb calling has been turned ON


(2.604077132966553, 15)

In [102]:
110 * (1/R_n**15) + 2.60

112.05137271119496

In [87]:
[node.derived_value for node in bm.lattice.layers[14]]

[171.95300071053308,
 159.13637211355226,
 147.27503925386117,
 136.29779854318545,
 126.13875359895198,
 116.73691966826433,
 108.03585753639491,
 99.983334722145,
 92.53101192622852,
 85.63415285042973,
 79.2513556455781,
 73.34430437622277,
 67.87753901004919,
 62.81824255128384,
 58.13604404024883]

In [69]:
vals, qs = choose_binomial(S0, u_n, d_n, q_n, 15)
vals

[178.77315075823685,
 165.44817784754298,
 153.11639044774742,
 141.70376083168938,
 131.1417789769353,
 121.36704130007331,
 112.32087004496371,
 103.94896104013372,
 96.20105771080371,
 89.0306493886385,
 82.3946921081774,
 76.25335021388379,
 70.56975722668103,
 65.30979453445644,
 60.44188657801225,
 55.936811302964905]

Q1. Compute the price of an American call option with strike $K = 110$ and maturity $T = .25$ years.

In [137]:
bm = BinomialModel(S0, u_n, d_n, q_n, R_n, n)
price, _ = bm.american_option(k=110, call=True)
price

2.604077132966553

Q2. Compute the price of an American put option with strike $K = 110$ and maturity $T = .25$ years.

In [138]:
bm = BinomialModel(S0, u_n, d_n, q_n, R_n, n)
price, _ = bm.american_option(k=110, call=False)
price

12.359784797284911

Q3. Should the option from Q2 be exercised early? If yes, during which period?

In [140]:
bm = BinomialModel(S0, u_n, d_n, q_n, R_n, n)
_, early_exercise = bm.american_option(k=110, call=False)
early_exercise

5